# IA - Projeto 01: Horário de turma 

# 1. Introdução
### Membros do grupo
- Hugo Ferreira Baptista — nº 23279
- Nuno da Cunha Faria Gajo — nº 23002


### Contexto e Objetivo do projeto
O objetivo deste projeto é para desenvolver um **agente inteligente** capaz de gerar **horários de turmas** que respeitem certas restrições tais como, disponibilidade de professores, salas e evitar conflitos de horários.  
O problema é formulado como um **CSP** utilizando Python e a biblioteca `python-constraint`.


In [ ]:
# Install contraint library
%pip install python-constraint

In [ ]:
# Import contraint library
from constraint import *

# 2. Design do Agente

Nesta seção, definimos o problema do horário de turmas como um **CSP**.

Cada variável representa uma **aula** e pode assumir um valor (bloco de tempo, sala, online) onde:
- `bloco de tempo` \ `block` ∈ [1; 20] (20 blocos por semana);
- `sala` \ `room` ∈ {S1, S2, S3, Lab01};
- `online` ∈ {Verdadeiro, Falso}.

Nós definimos:
- Variáveis e domínios;  
- Restrições rígidas (obrigatórias);
- Restrições flexíveis (opcionais, tratadas posteriormente);
- Heurísticas para melhorar o desempenho;

In [ ]:
# Exemplo de leitura do ficheiro e preparação dos dados:
with open('ClassTT_01_tiny.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line.strip())
        

In [ ]:
# =============================
#      DESIGN DO AGENTE
# =============================

from constraint import Problem

# --- Inicializar o problema CSP ---
problem = Problem()

# --- Recursos disponíveis ---
rooms = ['R1', 'R2', 'R3', 'Lab01']

# --- Cursos por turma ---
classes = {
    't01': ['UC11','UC12','UC13','UC14','UC15'],
    't02': ['UC21','UC22','UC23','UC24','UC25'],
    't03': ['UC31','UC32','UC33','UC34','UC35']
}

# --- Professores atribuídos ---
teachers = {
    'jo': ['UC11','UC21','UC22','UC31'],
    'mike': ['UC12','UC23','UC32'],
    'rob': ['UC13','UC14','UC24','UC33'],
    'sue': ['UC15','UC25','UC34','UC35']
}

# --- Restrições de disponibilidade (slots indisponíveis) ---
teacher_restrictions = {
    'mike': list(range(13,21)),               # slots 13–20
    'rob': list(range(1,5)),                  # slots 1–4
    'sue': [9,10,11,12,17,18,19,20]          # slots 9–12, 17–20
}

# --- Restrições de sala fixa ---
fixed_rooms = {
    'UC14': 'Lab01',
    'UC22': 'Lab01'
}

# --- Aulas online ---
online_courses = {
    'UC21': 2,    # 2ª aula é online
    'UC31': 2
}

# =============================
#     VARIÁVEIS E DOMÍNIOS
# =============================

# Cada curso tem 2 aulas semanais (UCxx_1 e UCxx_2)
for tclass, courses in classes.items():
    for course in courses:
        for i in [1,2]:
            var = f"{course}_{i}"
            # Cada valor é uma combinação (block, room, online)
            domain = [(block, room, online) for block in range(1,21)
                      for room in rooms for online in [False, True]]
            problem.addVariable(var, domain)

# =============================
#        HARD CONSTRAINTS
# =============================

#  Salas fixas — certas aulas só podem ocorrer na sala indicada
for course, room in fixed_rooms.items():
    for i in [1,2]:
        problem.addConstraint(lambda v, r=room: v[1] == r, (f"{course}_{i}",))

#  Disponibilidade de professores — evitar slots proibidos
for teacher, courses_list in teachers.items():
    unavailable = teacher_restrictions.get(teacher, [])
    for course in courses_list:
        for i in [1,2]:
            problem.addConstraint(lambda v, u=unavailable: v[0] not in u, (f"{course}_{i}",))

#  Conflitos de sala — duas aulas não podem ocorrer na mesma sala e horário
def no_room_conflict(a, b):
    return not (a[0] == b[0] and a[1] == b[1])

all_courses = [f"{c}_{i}" for cl in classes.values() for c in cl for i in [1,2]]
for i in range(len(all_courses)):
    for j in range(i+1, len(all_courses)):
        problem.addConstraint(no_room_conflict, (all_courses[i], all_courses[j]))

#  Aulas online — UC21_2 e UC31_2 devem ocorrer no mesmo dia
# (slots 1–4 = seg, 5–8 = ter, 9–12 = qua, 13–16 = qui, 17–20 = sex)
def same_day(a, b):
    return (a[0]-1)//4 == (b[0]-1)//4

problem.addConstraint(same_day, ("UC21_2", "UC31_2"))

# =============================
#  SOFT CONSTRAINTS (heurísticas)
# =============================
# Estas não invalidam soluções, mas servem para pontuação posterior:
# - Aulas da mesma disciplina em dias diferentes
# - Cada turma com até 4 dias de aulas
# - Aulas consecutivas no mesmo dia
# - Minimizar o nº de salas diferentes por turma

# (Implementadas mais tarde via função de avaliação)
# Get all the solutions.
#solutions = problem.getSolutions()

# Easier way to print and see all solutions
#for solution in solutions:
  #print(solution)
for teacher, subjects in teachers.items():
    print(f"{teacher}:")
    for uc in subjects:
        print(f"  - {uc}")
    

# 3. Agente em execução
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pulvinar semper augue vel porttitor. Aenean porta nulla congue sem venenatis, non varius lorem imperdiet. Aenean odio lacus, fringilla a libero ut, ullamcorper finibus lectus. Nunc ullamcorper eu urna volutpat egestas. Nunc vehicula maximus quam non lobortis. Ut eget dolor quis arcu vestibulum consectetur. Aliquam consequat lectus odio, in scelerisque orci lacinia sit amet.Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur vitae sapien urna. Ut at aliquet metus, vel porta metus. Curabitur quis tellus at sapien ullamcorper feugiat. Donec auctor, ante at ornare condimentum, magna mi ornare orci, vitae feugiat ligula sapien sed ex. Cras id interdum ipsum. Sed metus nibh, maximus at viverra at, pretium id est. Vivamus semper urna et libero gravida, euismod tincidunt sem hendrerit. Curabitur semper magna eu nisl laoreet congue. Curabitur viverra odio sed neque volutpat, volutpat feugiat nunc finibus. Donec mattis semper odio, eget tincidunt nulla convallis ut. Donec odio diam, facilisis vitae iaculis quis, gravida vel lacus. Sed finibus arcu sit amet dolor lobortis lacinia. Proin vestibulum lectus non lorem tempus aliquam. Cras ex nulla, condimentum sed mauris at, pulvinar finibus erat.

# 4. Conclusão
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pulvinar semper augue vel porttitor. Aenean porta nulla congue sem venenatis, non varius lorem imperdiet. Aenean odio lacus, fringilla a libero ut, ullamcorper finibus lectus. Nunc ullamcorper eu urna volutpat egestas. Nunc vehicula maximus quam non lobortis. Ut eget dolor quis arcu vestibulum consectetur. Aliquam consequat lectus odio, in scelerisque orci lacinia sit amet.